**Importing Libraries and Setting Up Figure Style**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# import ML
import lightgbm as lgb #gradient boosted decsion tree machine
from sklearn.model_selection import KFold, RandomizedSearchCV #cross validation
from sklearn.model_selection import train_test_split #split
from sklearn.metrics import r2_score, mean_squared_error #metrics

# set up plot styles
sns.set_theme()
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("talk")


Above we have imported all the modules needed for reading and cleaning the database, data viz and machine learning analysis of the data. We have chosen to use a light gradient boosted machine (lightgbm) to build our model as it has been shown to return models with low mean square error compared to decsion tree, random forest, SVM, kNN.We also set the appreance of plots at this step so our output has a consistent style. This is an important (and often overlooked!) point for convinvcing reports for stakeholders of both technical and non-technical backgrounds.
